In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [2]:
%load_ext autoreload
%autoreload 2

from os.path import join

import numpy as np

from pyspark.sql import SQLContext

In [63]:
path_root = "/home/tlin/notebooks/data"
path_pickle_train = join(
    path_root, "cache/two_domain/split_data/train")
path_pickle_test = join(
    path_root, "cache/two_domain/split_data/test")
path_pickle_baseline_sim = join(
    path_root, "cache/two_domain/item_based_sim/base_sim")
path_pickle_extended_sim = join(
    path_root, "cache/two_domain/extend_sim/extendsim")
path_pickle_private_mapped_sim = join(
    path_root, "cache/two_domain/private_mapping/privatemap")
path_pickle_nonprivate_mapped_sim = join(
    path_root, "cache/two_domain/private_mapping/nonprivatemap")
path_pickle_userbased_alterEgo = join(
    path_root, "cache/two_domain/cross_sim/user_based_alterEgo")
path_pickle_itembased_alterEgo = join(
    path_root, "cache/two_domain/cross_sim/item_based_alterEgo")
path_pickle_alterEgo_sim = join(
    path_root, "cache/two_domain/cross_sim/targetdomain_sim")
path_pickle_private_policy_sim = join(
    path_root, "cache/two_domain/private_policy/policy_sim")

In [64]:
train_dataRDD = sc.pickleFile(path_pickle_train).cache()
test_dataRDD = sc.pickleFile(path_pickle_test).cache()
alterEgo_sim = sc.pickleFile(path_pickle_alterEgo_sim).cache()
sim_pair = sc.pickleFile(path_pickle_private_policy_sim).cache()

In [65]:
item_based_alterEgo = sc.pickleFile(path_pickle_itembased_alterEgo).cache()
user_based_alterEgo = sc.pickleFile(path_pickle_userbased_alterEgo).cache()

user_based_dict_bd = sc.broadcast(user_based_alterEgo.collectAsMap())
item_based_dict_bd = sc.broadcast(item_based_alterEgo.collectAsMap())

sim_pair_dict_bd  = sc.broadcast(sim_pair.collectAsMap())

In [74]:
    def get_info(dataRDD):
        """get the information of RDD, either item or user.
        Args:
            dataRDD could either be userRDD or itemRDD.
            userRDD: (uid, (iid, rating, rating time)*)
            itemRDD: (iid, (uid, rating, rating time)*)
        Returns:
            info of the input RDD:
                (uid, (average, norm2, count))* or
                (iid, (average, norm2, count))* or
        """
        def norm2(ratings):
            """calculate the norm 2 of input ratings.
            Args:
                ratings: (iid, rating, rating time)*
            Returns:
                norm of ratings.
            """
            return np.sqrt(np.sum([rating[1] ** 2 for rating in ratings]))

        def average(ratings):
            """calculate the average of the ratings.
            Args:
                ratings: (iid, rating, rating time)*
            Returns:
                average of ratings.
            """
            return 1.0 * np.average([rating[1] for rating in ratings])

        def helper(line):
            """a helper function."""
            uid, ratings = line
            return uid, (average(ratings), norm2(ratings), len(ratings))
        return dataRDD.map(helper)


user_info = get_info(user_based_alterEgo)
item_info = get_info(item_based_alterEgo)

user_info_bd = sc.broadcast(user_info.collectAsMap())
item_info_bd = sc.broadcast(item_info.collectAsMap())

In [85]:
sim_pair_dict_keys = set(sim_pair_dict_bd.value.keys())
test = test_dataRDD.filter(lambda line: line[0] in sim_pair_dict_keys)

In [98]:
t = test_dataRDD.take(1)[0]
uid, pairs = t
pair = pairs[0]
iid, real_rating = pair[0], pair[1]
iid, real_rating

('T:B003T2YI8G', 2.0)

In [99]:
item_bd = item_info_bd

iid_neighbors = [(i[0], i[1], rating_bd.value[i[0]]) for i in sim_bd.value[iid]]
average_iid_rating = item_bd.value[iid][0]

KeyError: 'T:B003T2YI8G'

In [101]:
sim_pair_dict_bd.value

{'A1AFXJ8U72MD6L': [('ADR6SKFTGZAYX', -0.013928192779141839)],
 'A3HSC9QAAH6F48': [('A127W6EUS9C5XT', -0.15783375805756436)],
 'A11PK2MO25K5QW': [('A3GZHT1PNYP8G6', 0.030766462013074654)],
 'A1SBSKH1JJ10K5': [('A17LEYMZI8OYOQ', 0.013281576881580364)],
 'A1H9NI4ELHO03K': [('A13HLHUD9K0T3K', 0.10385365056806109)],
 'AYKN84MIK63TP': [('A1F1JMFGPVSR66', -0.0002630102345828765)],
 'A3MO7GWJ5HRIAZ': [('A2LSP1V2R26HUV', 0.027699923930809493)],
 'A2BG2FWZE87ECG': [('AMHNY61SWKH6E', -0.00092886103496397617)],
 'A2NFC5XBCIJZ6O': [('A2IIAQ7VVAHXTD', 0.0022639514305463973)],
 'A170UXQQ56AHO4': [('A22Z554ZQ8NFPC', 0.0039460461445237696)],
 'A1ZNROXU2AQTXC': [('A3BFQGSCCQ8U5O', 0.00049454512700914375)],
 'A3QGK33FM8YP7Y': [('AS7LF9YEEYYC1', -0.0044031142002224199)],
 'A30MZYD9VMCODJ': [('ASRI9DDKDTPG5', 0.00024629917150337504)],
 'A2ZMT35WPJLXA2': [('A3E7JN81UHJRQ9', 0.083719998191654474)],
 'A1S9IZV0EJWWY7': [('A25Q6V752YWUXH', 0.12739764978085077)],
 'A1QAT0NMU0EHM8': [('A2B3Z2QHW3V35S', 0.0054410